In [1]:
"""
Python Installation Helper for Smart Traffic Management System
This script handles common installation errors and provides fixes
"""

import subprocess
import sys
import os
import platform
from pathlib import Path

class TrafficSystemInstaller:
    def __init__(self):
        self.os_type = platform.system()
        self.python_version = sys.version_info
        self.errors = []
        self.warnings = []

    def check_python_version(self):
        """Check if Python version is compatible"""
        print("Checking Python version...")
        if self.python_version.major < 3 or (self.python_version.major == 3 and self.python_version.minor < 7):
            self.errors.append("Python 3.7+ is required. Current version: {}.{}".format(
                self.python_version.major, self.python_version.minor))
            return False
        print(f"✓ Python {self.python_version.major}.{self.python_version.minor} detected")
        return True

    def install_package(self, package_name, extra_args=None):
        """Install a Python package with error handling"""
        try:
            print(f"Installing {package_name}...")
            cmd = [sys.executable, "-m", "pip", "install"]

            if extra_args:
                cmd.extend(extra_args)
            cmd.append(package_name)

            result = subprocess.run(cmd, capture_output=True, text=True)

            if result.returncode == 0:
                print(f"✓ {package_name} installed successfully")
                return True
            else:
                print(f"✗ Failed to install {package_name}")
                self.errors.append(f"{package_name}: {result.stderr}")
                return False
        except Exception as e:
            print(f"✗ Error installing {package_name}: {e}")
            self.errors.append(f"{package_name}: {str(e)}")
            return False

    def install_sumo(self):
        """Install SUMO based on operating system"""
        print("\n" + "="*50)
        print("Installing SUMO Traffic Simulator...")
        print("="*50)

        if self.os_type == "Linux":
            print("For Ubuntu/Debian, run these commands in terminal:")
            print("  sudo apt-get update")
            print("  sudo apt-get install -y sumo sumo-tools sumo-doc")
            print("\nIf that doesn't work, try:")
            print("  sudo add-apt-repository ppa:sumo/stable")
            print("  sudo apt-get update")
            print("  sudo apt-get install -y sumo sumo-tools sumo-doc")

        elif self.os_type == "Darwin":  # macOS
            print("For macOS, run these commands in terminal:")
            print("  brew tap dlr-ts/sumo")
            print("  brew install sumo")

        elif self.os_type == "Windows":
            print("For Windows:")
            print("1. Download SUMO from: https://sumo.dlr.de/docs/Downloads.php")
            print("2. Run the installer")
            print("3. Add SUMO to your PATH environment variable")
            print("\nOr use Chocolatey:")
            print("  choco install sumo")

        return True

    def install_python_packages(self):
        """Install all required Python packages"""
        print("\n" + "="*50)
        print("Installing Python Packages...")
        print("="*50)

        # Update pip first
        print("Updating pip...")
        subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
                      capture_output=True)

        # Core packages
        packages = [
            ("numpy", None),
            ("gymnasium", None),
            ("stable-baselines3", None),
            ("tensorboard", None),
            ("matplotlib", None),
            ("pandas", None),
        ]

        # Install packages
        failed_packages = []
        for package, args in packages:
            if not self.install_package(package, args):
                failed_packages.append(package)

        # Try to install SUMO-RL (often problematic)
        print("\nInstalling SUMO-RL (this might take a while)...")

        # First, try installing traci separately
        if not self.install_package("traci"):
            print("Warning: traci installation failed")
            self.warnings.append("traci might not be properly installed")

        # Try installing sumo-rl
        if not self.install_package("sumo-rl"):
            print("Trying alternative installation method for sumo-rl...")
            # Try without dependencies
            if not self.install_package("sumo-rl", ["--no-deps"]):
                self.errors.append("sumo-rl installation failed. You may need to install it manually.")
                failed_packages.append("sumo-rl")

        # Optional: Install Ray RLlib
        install_ray = input("\nDo you want to install Ray[RLlib]? (y/n, default=n): ").lower()
        if install_ray == 'y':
            print("Installing Ray[RLlib] (this is a large package)...")
            if not self.install_package("ray[rllib]"):
                self.warnings.append("Ray[RLlib] installation failed (optional)")

        return len(failed_packages) == 0

    def setup_environment_variables(self):
        """Setup SUMO_HOME environment variable"""
        print("\n" + "="*50)
        print("Setting up Environment Variables...")
        print("="*50)

        sumo_paths = []

        if self.os_type == "Linux":
            sumo_paths = [
                "/usr/share/sumo",
                "/usr/local/share/sumo",
                "/opt/sumo/share/sumo"
            ]
        elif self.os_type == "Darwin":  # macOS
            sumo_paths = [
                "/usr/local/opt/sumo/share/sumo",
                "/opt/homebrew/opt/sumo/share/sumo"
            ]
        elif self.os_type == "Windows":
            sumo_paths = [
                "C:\\Program Files (x86)\\Eclipse\\Sumo",
                "C:\\Program Files\\Eclipse\\Sumo",
                "C:\\Sumo"
            ]

        # Find SUMO installation
        sumo_home = None
        for path in sumo_paths:
            if os.path.exists(path):
                sumo_home = path
                break

        if not sumo_home:
            # Try to find SUMO using which/where command
            try:
                if self.os_type == "Windows":
                    result = subprocess.run(["where", "sumo"], capture_output=True, text=True)
                else:
                    result = subprocess.run(["which", "sumo"], capture_output=True, text=True)

                if result.returncode == 0:
                    sumo_bin = result.stdout.strip()
                    sumo_home = str(Path(sumo_bin).parent.parent / "share" / "sumo")
            except:
                pass

        if sumo_home and os.path.exists(sumo_home):
            print(f"Found SUMO at: {sumo_home}")

            # Set environment variable
            os.environ['SUMO_HOME'] = sumo_home
            os.environ['PATH'] = os.path.join(sumo_home, 'bin') + os.pathsep + os.environ.get('PATH', '')

            print("\nAdd these lines to your shell configuration file:")
            if self.os_type == "Windows":
                print(f"  setx SUMO_HOME \"{sumo_home}\"")
                print(f"  setx PATH \"%PATH%;{os.path.join(sumo_home, 'bin')}\"")
            else:
                print(f"  export SUMO_HOME=\"{sumo_home}\"")
                print(f"  export PATH=\"$SUMO_HOME/bin:$PATH\"")
                print("\nAdd to ~/.bashrc or ~/.zshrc for permanent setup")

            return True
        else:
            print("✗ Could not find SUMO installation")
            print("Please install SUMO first and set SUMO_HOME manually")
            self.warnings.append("SUMO_HOME not set - manual configuration required")
            return False

    def verify_installation(self):
        """Verify all components are installed correctly"""
        print("\n" + "="*50)
        print("Verifying Installation...")
        print("="*50)

        all_good = True

        # Check Python packages
        required_packages = {
            'numpy': 'NumPy',
            'gymnasium': 'Gymnasium',
            'stable_baselines3': 'Stable-Baselines3',
            'tensorboard': 'TensorBoard',
            'matplotlib': 'Matplotlib',
            'pandas': 'Pandas'
        }

        print("\nPython Packages:")
        for module, name in required_packages.items():
            try:
                __import__(module)
                print(f"  ✓ {name}")
            except ImportError:
                print(f"  ✗ {name}")
                all_good = False

        # Check SUMO-RL
        try:
            import sumo_rl
            print("  ✓ SUMO-RL")
        except ImportError:
            print("  ✗ SUMO-RL (optional but recommended)")
            self.warnings.append("SUMO-RL not available")

        # Check SUMO
        print("\nSUMO Installation:")
        if 'SUMO_HOME' in os.environ:
            print(f"  ✓ SUMO_HOME = {os.environ['SUMO_HOME']}")
        else:
            print("  ✗ SUMO_HOME not set")
            all_good = False

        # Try to import traci
        try:
            import traci
            print("  ✓ TraCI (SUMO Python API)")
        except ImportError:
            print("  ✗ TraCI not available")
            self.warnings.append("TraCI not available - SUMO integration may not work")

        return all_good

    def fix_common_errors(self):
        """Provide fixes for common installation errors"""
        print("\n" + "="*50)
        print("Common Error Fixes")
        print("="*50)

        fixes = {
            "gymnasium": [
                "If gymnasium fails, try:",
                "  pip install gym",
                "  pip install gymnasium --upgrade"
            ],
            "stable-baselines3": [
                "If stable-baselines3 fails, try:",
                "  pip install torch",
                "  pip install stable-baselines3 --no-deps",
                "  pip install stable-baselines3"
            ],
            "sumo-rl": [
                "If sumo-rl fails, try:",
                "  pip install traci",
                "  pip install sumo-rl --no-deps",
                "Or clone from GitHub:",
                "  git clone https://github.com/LucasAlegre/sumo-rl",
                "  cd sumo-rl && pip install -e ."
            ],
            "SUMO": [
                "If SUMO is not found:",
                "1. Ensure SUMO is installed",
                "2. Find SUMO installation directory",
                "3. Set SUMO_HOME environment variable",
                "4. Add SUMO/bin to PATH"
            ]
        }

        for error_type, fix_steps in fixes.items():
            print(f"\n{error_type}:")
            for step in fix_steps:
                print(f"  {step}")

    def create_test_script(self):
        """Create a test script to verify the installation"""
        test_script = '''#!/usr/bin/env python3
"""
Test script for Smart Traffic Management System
Run this to verify your installation is working
"""

import sys
import os

def test_imports():
    """Test if all required packages can be imported"""
    print("Testing package imports...")

    packages = [
        'numpy',
        'gymnasium',
        'stable_baselines3',
        'matplotlib',
        'pandas',
        'tensorboard'
    ]

    failed = []
    for package in packages:
        try:
            __import__(package)
            print(f"  ✓ {package}")
        except ImportError as e:
            print(f"  ✗ {package}: {e}")
            failed.append(package)

    # Test SUMO-RL
    try:
        import sumo_rl
        print(f"  ✓ sumo-rl")
    except ImportError:
        print(f"  ⚠ sumo-rl (optional)")

    # Test TraCI
    try:
        import traci
        print(f"  ✓ traci")
    except ImportError:
        print(f"  ⚠ traci (needed for SUMO)")

    return len(failed) == 0

def test_sumo():
    """Test SUMO configuration"""
    print("\\nTesting SUMO configuration...")

    if 'SUMO_HOME' in os.environ:
        print(f"  ✓ SUMO_HOME = {os.environ['SUMO_HOME']}")
        return True
    else:
        print("  ✗ SUMO_HOME not set")
        return False

def test_rl_environment():
    """Test creating a simple RL environment"""
    print("\\nTesting RL environment creation...")

    try:
        import gymnasium as gym
        from stable_baselines3 import PPO

        # Create a simple environment
        env = gym.make('CartPole-v1')
        print("  ✓ Gymnasium environment created")

        # Create a PPO model
        model = PPO('MlpPolicy', env, verbose=0)
        print("  ✓ PPO model created")

        env.close()
        return True
    except Exception as e:
        print(f"  ✗ Failed: {e}")
        return False

if __name__ == "__main__":
    print("="*50)
    print("Smart Traffic Management System - Installation Test")
    print("="*50)

    results = []
    results.append(test_imports())
    results.append(test_sumo())
    results.append(test_rl_environment())

    print("\\n" + "="*50)
    if all(results):
        print("✅ All tests passed! Your system is ready.")
    else:
        print("❌ Some tests failed. Please check the errors above.")
    print("="*50)
'''

        with open('test_installation.py', 'w') as f:
            f.write(test_script)

        print("\nCreated test_installation.py")
        print("Run it with: python test_installation.py")

    def run(self):
        """Run the complete installation process"""
        print("="*60)
        print(" Smart Traffic Management System - Installation Helper")
        print(" Problem ID: 25050 - Government of Odisha")
        print("="*60)

        # Check Python version
        if not self.check_python_version():
            print("\n❌ Please install Python 3.7 or higher")
            return False

        # Install SUMO instructions
        self.install_sumo()

        # Install Python packages
        self.install_python_packages()

        # Setup environment variables
        self.setup_environment_variables()

        # Verify installation
        success = self.verify_installation()

        # Show common fixes
        if not success or self.errors or self.warnings:
            self.fix_common_errors()

        # Create test script
        self.create_test_script()

        # Final summary
        print("\n" + "="*60)
        print("Installation Summary")
        print("="*60)

        if self.errors:
            print("\n❌ Errors encountered:")
            for error in self.errors:
                print(f"  - {error}")

        if self.warnings:
            print("\n⚠ Warnings:")
            for warning in self.warnings:
                print(f"  - {warning}")

        if success and not self.errors:
            print("\n✅ Installation completed successfully!")
            print("Next steps:")
            print("1. Run: python test_installation.py")
            print("2. Start training with the main script")
        else:
            print("\n⚠ Installation completed with issues")
            print("Please fix the errors above before proceeding")

        return success

if __name__ == "__main__":
    installer = TrafficSystemInstaller()
    installer.run()

 Smart Traffic Management System - Installation Helper
 Problem ID: 25050 - Government of Odisha
Checking Python version...
✓ Python 3.12 detected

Installing SUMO Traffic Simulator...
For Windows:
1. Download SUMO from: https://sumo.dlr.de/docs/Downloads.php
2. Run the installer
3. Add SUMO to your PATH environment variable

Or use Chocolatey:
  choco install sumo

Installing Python Packages...
Updating pip...
Installing numpy...
✓ numpy installed successfully
Installing gymnasium...
✓ gymnasium installed successfully
Installing stable-baselines3...
✓ stable-baselines3 installed successfully
Installing tensorboard...
✓ tensorboard installed successfully
Installing matplotlib...
✓ matplotlib installed successfully
Installing pandas...
✓ pandas installed successfully

Installing SUMO-RL (this might take a while)...
Installing traci...
✓ traci installed successfully
Installing sumo-rl...
✓ sumo-rl installed successfully

Setting up Environment Variables...
Found SUMO at: C:\Program Files